## Backtesting

In [1]:
import os

os.chdir("..")

### Seeding our portfolio
Before starting a backtest we'll generally want to initialise our assets and seed a portfolio.

In [2]:
from pytrade.assets import reset, Cash, Stock, Portfolio
from pytrade.backtest import Backtest
from pytrade.strategy import Strategy

In [3]:
reset()
usd = Cash("USD")
stock = Stock("XXX US", 2.50, currency_code="USD")
portfolio = Portfolio("USD")

portfolio.transfer(usd, 1000)
print(portfolio)

Portfolio('USD'):
Cash('USD', 1.0, currency_code='USD'): 1,000


### Setting portfolio compliance and broker execution strategy.

Recall from a previous notebook that the portfolio holds both compliance and broker objects. By default the compliance checks are empty and the portfolio broker executes trades at the last available price with no fee. 

In [4]:
portfolio.compliance

In [5]:
portfolio.broker

If you want to configure these it should be done before running the backtest. Suppose I want to limit the portfolio holding of 'XXX US' to 1000 shares in our portfolio.

In [6]:
from pytrade.compliance import Compliance, UnitLimit

portfolio.compliance = Compliance().add_rule(
    UnitLimit(stock, 1000)
)

len(portfolio.compliance)  # there is now one compliance rule in place

1

Also configure the portfolio broker: execution slippage of 50bps for a fixed USD 20 charge.

In [7]:
from pytrade.broker import (
    Broker,
    FillAtLastWithSlippage,
    FixedRatePlusPercentage,
)

portfolio.broker = Broker(
    execution_strategy=FillAtLastWithSlippage(0.005),
    charges_strategy = FixedRatePlusPercentage(20, 0.0, currency_code="USD")
)

### Setting the backtest strategy

All backtests should have a strategy that inherits from pytrade.strategy.Strategy and implements a generate_trades method. Returned trades can either be None, a single trade instance or a list of trades.

In [8]:
from pytrade import Trade
from pytrade.strategy import Strategy

class BuyAndHold(Strategy):
    def generate_trades(self):
        return Trade(portfolio, stock, 500)


In this case the strategy always returns a single order to buy 500 shares of stock.

### Creating the backtest instance
Backtests can be initialised by passing in an instance of the strategy under test.

In [9]:
from pytrade.backtest import Backtest

backtest = Backtest(BuyAndHold())

backtest

### Keeping a record of the backtest history
In order to maintain backtest records over the life of the backtest we'll need to create a history instance.

In [10]:
from pytrade.history import History

history = History(
    portfolios=portfolio,
    backtest=backtest,
)

We can get a data frame of history throughout the backtest. As we have not yet run the backtest this is currently empty.

In [11]:
df = history.get()

df

""


### Loading events for the backtest
In future notebooks we'll demonstrate how to load events in bulk from yahoo finance or a data frame. However, for now we're going to manually load events to keep the example small. The backtest instance has a load_event method.

In [12]:
backtest.load_event

<bound method Backtest.load_event of <pytrade.backtest.Backtest object at 0x000002C2E394B6A0>>

In [13]:
from datetime import datetime
from pytrade.events import AssetPriceEvent

events = [
    AssetPriceEvent(stock, datetime(2020, 10, 1), 2.50),
    AssetPriceEvent(stock, datetime(2020, 10, 2), 2.55),
    AssetPriceEvent(stock, datetime(2020, 10, 3), 2.60),
    AssetPriceEvent(stock, datetime(2020, 10, 4), 2.65),
    AssetPriceEvent(stock, datetime(2020, 10, 5), 2.70),
]

[backtest.load_event(event) for event in events]
backtest.num_events_loaded

5

So we've loaded 5 price events for stock 'XXX US' spanning 1st Oct 2020 -> 5th Oct 2020.

We're not ready to run our backtest.

In [14]:
backtest.run()

In [15]:
history.get()

,Portfolio,Portfolio_USD,Portfolio_XXX US,USD,XXX US
2020-10-01,973.750,-276.250,500.0,1.0,2.50
2020-10-02,972.375,-1577.625,1000.0,1.0,2.55
2020-10-03,1022.375,-1577.625,1000.0,1.0,2.60
2020-10-04,1072.375,-1577.625,1000.0,1.0,2.65
2020-10-05,1122.375,-1577.625,1000.0,1.0,2.70


Let's run through the sequence of events:
-  The portfolio has a starting value of USD 1000.


-  Upon loading events for 1st Oct the strategy generates a buy for 500 shares of 'XXX US' @2.50. This is executed by the broker with slippage of 0.5% and a fixed charge of USD 20. Broker charges + slippage = 20 + 500 x 2.50 x 0.5% = USD 26.25. The total outlay is 500 x (2.50 x (1+0.5%)) + 20 = 1276.25. We are then in a short USD position of starting usd - outlay = (1000 - 1276.25) = -276.25. The value of our stock is 500 shares at 2.50 = 1250, so we've lost 26.25 in broker charges and slippage. The total portfolio value is then 973.75.


-  On 2nd Oct another 500 shares is purchased and we arrive at a total position of 1000 shares in 'XXX US'. This is the maximum position that our compliance will allow. Further trades to buy this stock will get generated by the strategy after each event time but they'll get kicked back by compliance and not sent to the portfolio broker for execution.


-  This position of 1000 shares is then held to until the end of the backtest and the portfolio value increases with the share price.

In [16]:
20 + 500 * 2.50 * 0.005  # broker charges + slippage for first trade @2.50

26.25

In [17]:
20 + 500 * 2.55 * 0.005  # broker charges + slippage for second trade @2.55

26.375

In [18]:
26.25 + 26.375  # total broker charges and slippage

52.625

In [19]:
1000 * 2.70 - (500 * 2.50 + 500 * 2.55)  # stock gains

175.0

In [20]:
175.0 - 52.625  # portfolio gains

122.375

### Putting this all together

In [21]:
from datetime import datetime
from pytrade import Trade
from pytrade.assets import reset, Cash, Stock, Portfolio
from pytrade.compliance import Compliance, UnitLimit
from pytrade.backtest import Backtest
from pytrade.strategy import Strategy
from pytrade.events import AssetPriceEvent
from pytrade.broker import (
    Broker,
    FillAtLastWithSlippage,
    FixedRatePlusPercentage,
)


# initialise portfolio
reset()
usd = Cash("USD")
stock = Stock("XXX US", 2.50, currency_code="USD")
portfolio = Portfolio("USD")
portfolio.transfer(usd, 1000)

# set portfolio compliance and broker
portfolio.compliance = Compliance().add_rule(
    UnitLimit(stock, 1000)
)

portfolio.broker = Broker(
    execution_strategy=FillAtLastWithSlippage(0.005),
    charges_strategy = FixedRatePlusPercentage(20, 0.0, currency_code="USD")
)


# create our backtest strategy
class BuyAndHold(Strategy):
    def generate_trades(self):
        return Trade(portfolio, stock, 500)


# initialise our backtest and history
backtest = Backtest(BuyAndHold())
history = History(
    portfolios=portfolio,
    backtest=backtest,
)

# load events
events = [
    AssetPriceEvent(stock, datetime(2020, 10, 1), 2.50),
    AssetPriceEvent(stock, datetime(2020, 10, 2), 2.55),
    AssetPriceEvent(stock, datetime(2020, 10, 3), 2.60),
    AssetPriceEvent(stock, datetime(2020, 10, 4), 2.65),
    AssetPriceEvent(stock, datetime(2020, 10, 5), 2.70),
]

[backtest.load_event(event) for event in events]

# run the backtest
backtest.run()

In [22]:
df = history.get()
df

,Portfolio,Portfolio_USD,Portfolio_XXX US,USD,XXX US
2020-10-01,973.750,-276.250,500.0,1.0,2.50
2020-10-02,972.375,-1577.625,1000.0,1.0,2.55
2020-10-03,1022.375,-1577.625,1000.0,1.0,2.60
2020-10-04,1072.375,-1577.625,1000.0,1.0,2.65
2020-10-05,1122.375,-1577.625,1000.0,1.0,2.70


In [24]:
import cufflinks as cf

columns = ["Portfolio", "XXX US"]

df[columns].iplot(
    secondary_y="XXX US",
    title="Portfolio Value",
)

***